## Implementing Byte Pair Encoding

In [29]:
import os
os.environ["PYTHONIOENCODING"] = "utf-8"

In [32]:
print("नमस्ते")

नमस्ते


In [1]:
text = 'Every breath you take'
print(text)

Every breath you take


In [2]:
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'Every breath you take')


In [3]:
ids = list(byte_ary)
ids

[69,
 118,
 101,
 114,
 121,
 32,
 98,
 114,
 101,
 97,
 116,
 104,
 32,
 121,
 111,
 117,
 32,
 116,
 97,
 107,
 101]

In [4]:
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode(text)

[6109, 8033, 345, 1011]

In [7]:
for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")

0: !
1: "
2: #
3: $
4: %
5: &
6: '
7: (
8: )
9: *
10: +
11: ,
12: -
13: .
14: /
15: 0
16: 1
17: 2
18: 3
19: 4
20: 5
21: 6
22: 7
23: 8
24: 9
25: :
26: ;
27: <
28: =
29: >
30: ?
31: @
32: A
33: B
34: C
35: D
36: E
37: F
38: G
39: H
40: I
41: J
42: K
43: L
44: M
45: N
46: O
47: P
48: Q
49: R
50: S
51: T
52: U
53: V
54: W
55: X
56: Y
57: Z
58: [
59: \
60: ]
61: ^
62: _
63: `
64: a
65: b
66: c
67: d
68: e
69: f
70: g
71: h
72: i
73: j
74: k
75: l
76: m
77: n
78: o
79: p
80: q
81: r
82: s
83: t
84: u
85: v
86: w
87: x
88: y
89: z
90: {
91: |
92: }
93: ~
94: �
95: �
96: �
97: �
98: �
99: �
100: �
101: �
102: �
103: �
104: �
105: �
106: �
107: �
108: �
109: �
110: �
111: �
112: �
113: �
114: �
115: �
116: �
117: �
118: �
119: �
120: �
121: �
122: �
123: �
124: �
125: �
126: �
127: �
128: �
129: �
130: �
131: �
132: �
133: �
134: �
135: �
136: �
137: �
138: �
139: �
140: �
141: �
142: �
143: �
144: �
145: �
146: �
147: �
148: �
149: �
150: �
151: �
152: �
153: �
154: �
155: �
156: �
157: �
158:

### Algorithm
- We go step by step, find the frequency of the most repeated pairs
- We assign an ID to them
- We do the same again, find the next most common repeated pairs
- Assign IDs to them.

In [46]:
text_file_path = 'hindi_sample.txt'
f = open(text_file_path, "r")
sample = f.read()

### Building a Hindi Tokenizer

In [50]:
def preprocess_text(text):
    # Remove unnecessary whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters but keep Hindi characters
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
    return text.strip()

In [51]:
sample_preprocessed = preprocess_text(sample)
sample_preprocessed

'गोपी प्रेम हनुमान प्रसाद पोद्दार द्वारा लिखी गई एक अद्वितीय धार्मिक और भक्ति साहित्य की पुस्तक है। यह पुस्तक भगवान श्रीकृष्ण और गोपियों के बीच के प्रेम और भक्ति के अद्वितीय और अनमोल संबंधों को सजीव रूप में प्रस्तुत करती है। गोपियों का भगवान कृष्ण के प्रति असीम प्रेम भक्ति और समर्पण इस पुस्तक का मुख्य विषय है।हनुमान प्रसाद पोद्दार ने इस पुस्तक में गोपियों की निश्छल भक्ति उनकी तन्मयता और भगवान कृष्ण के प्रति उनके प्रेम का बहुत ही सुंदर और मार्मिक वर्णन किया है। पुस्तक में वर्णित भावनाएँ और भक्ति रस पाठकों को गहरे आध्यात्मिक और भावनात्मक स्तर पर छू जाती हैं। गोपी प्रेम न केवल भक्तों को भगवान श्रीकृष्ण की भक्ति की महिमा से अवगत कराती है बल्कि उन्हें भक्ति मार्ग पर चलने के लिए प्रेरित भी करती है। यह पुस्तक भगवान श्रीकृष्ण के प्रति गोपियों की अनन्य भक्ति को जीवंत करती है और पाठकों को भक्ति और प्रेम के वास्तविक अर्थ से अवगत कराती है। गोपी प्रेम हर उस व्यक्ति के लिए एक अनमोल धरोहर है जो भगवान श्रीकृष्ण के प्रति गहरी भक्ति और प्रेम को समझना और अनुभव करना चाहता है।'

In [60]:
processed_text = []
for i , char in enumerate(sample_preprocessed[:50]):
    if char == " " and i!=0:
        processed_text.append("</w>")
    if char != " ":
        processed_text.append(char)
processed_text = "".join(processed_text)

In [61]:
processed_text

'गोपी</w>प्रेम</w>हनुमान</w>प्रसाद</w>पोद्दार</w>द्वारा</w>लिखी</w>गई</w>एक'

In [62]:
unique_chars = [chr(i) for i in range(256)]
unique_chars

['\x00',
 '\x01',
 '\x02',
 '\x03',
 '\x04',
 '\x05',
 '\x06',
 '\x07',
 '\x08',
 '\t',
 '\n',
 '\x0b',
 '\x0c',
 '\r',
 '\x0e',
 '\x0f',
 '\x10',
 '\x11',
 '\x12',
 '\x13',
 '\x14',
 '\x15',
 '\x16',
 '\x17',
 '\x18',
 '\x19',
 '\x1a',
 '\x1b',
 '\x1c',
 '\x1d',
 '\x1e',
 '\x1f',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '\x7f',
 '\x80',
 '\x81',
 '\x82',
 '\x83',
 '\x84',
 '\x85',
 '\x86',
 '\x87',
 '\x88',
 '\x89',
 '\x8a',
 '\x8b',
 '\x8c',
 '\x8d',
 '\x8e',
 '\

In [8]:
class BPETokenizerHindi:
    def __init__(self):
        self.vocab = {}
        self.token_to_id = {}
        self.id_to_token = {}
        self.bpe_merges = 0

    def preprocess_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        # Remove special characters but keep Hindi characters
        text = re.sub(r'[^\u0900-\u097F\s]', '', text)
        return text.strip()

    def train(self, text):
        ## Training Loop
        processed_text = []
        for i , char in enumerate(text):
            if char == " " and i!=0:
                processed_text.append("</w>")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)
            
        unique_chars = [chr(i) for i in range(256)]

        if "</w>" not in unique_chars:
            unique_chars.append("</w>")

        # Creating Vocab
        self.vocab = {i:char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        tokens_ids = [self.inverse_vocab[char] for char in processed_text]

        